이미지 관련 코드

In [3]:
!pip install opencv-python

  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (35.4 MB)


In [15]:
import os
import cv2
import numpy as np
import random
import math

In [10]:
import cv2
import random
import math

In [67]:
import os
import cv2

# 원본 이미지가 있는 디렉토리
input_dir = "/Users/joseohyeon/Downloads/big6/data/FER2013Valid"

# 이미지를 저장할 디렉토리
output_dir = "/Users/joseohyeon/Downloads/big6/data1/FER2013Valid"

# 저장할 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내의 모든 이미지 파일에 대해 반복
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # 이미지 파일만 처리
        # 이미지를 읽어옴
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)
        
        # 이미지 반전
        flip_img = cv2.flip(img, 1)

        # 랜덤한 각도로 회전 (0도는 제외)
        angle = random.choice([-20, -19, -18, -17, -16, 16, 17, 18, 19, 20])  # -20에서 -15 또는 15에서 20 중에서 선택
        if angle != 0:
            # 회전한 이미지의 외접 사각형 좌표 계산
            radians = math.radians(abs(angle))
            cos_theta = math.cos(radians)
            sin_theta = math.sin(radians)   

            rows, cols, _ = img.shape

            new_width = int((cols * cos_theta) + (rows * sin_theta))
            new_height = int((cols * sin_theta) + (rows * cos_theta))

            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotate_img = cv2.warpAffine(img, M, (new_width, new_height))
            
            # 회전된 이미지를 내부에 있는 영역을 기준으로 확대
            enlarged_img = cv2.resize(rotate_img, (new_width, new_height))
            
            # 확대된 이미지에서 48x48 크기로 잘라냄
            cropped_img = enlarged_img[int(max(new_width, new_height)/2) - 24:int(max(new_width, new_height)/2)+12, int(max(new_width, new_height)/2)-24:int(max(new_width, new_height)/2)+12]
    
            # 확대된 이미지를 저장
            enlarged_output_path = os.path.join(output_dir, f"r_{filename}")
            cv2.imwrite(enlarged_output_path, cropped_img)
    
        # 반전된 이미지를 저장
        output_path = os.path.join(output_dir, f"f_{filename}")
        cv2.imwrite(output_path, flip_img)
print("이미지 증강 및 저장 완료")


이미지 증강 및 저장 완료


In [36]:
import pandas as pd

# 기존 CSV 파일 경로
csv_file_path = './fer2013new.csv'

# 중간 CSV
middle_file_path = './fer2013new_.csv'

# 새로운 CSV 파일 경로
new_csv_file_path = './fer2013new__.csv'

# CSV 파일을 읽어옴
df = pd.read_csv(csv_file_path)
mdf = pd.read_csv(middle_file_path)

# 새로운 데이터를 담을 빈 DataFrame 생성
new_df = pd.DataFrame(columns=df.columns)

# 기존 데이터프레임을 복사하여 r_를 추가한 새로운 행을 생성하여 새로운 데이터프레임에 추가
for index, row in df.iterrows():
    if not pd.isnull(row['Image name']):  # 'Image name' 열이 NaN이 아닌 경우에만 처리
        new_row = row.copy()
        # new_row['Image name'] = 'r_' + new_row['Image name']
        new_row['Image name'] = 'f_' + new_row['Image name']
        new_df = pd.concat([new_df, new_row.to_frame().transpose()], ignore_index=True)

# 기존 데이터프레임과 새로운 데이터프레임을 합쳐서 새로운 CSV 파일로 저장
final_df = pd.concat([mdf, new_df])
final_df.to_csv(new_csv_file_path, index=False)

print("CSV 파일 복사 및 추가 완료")


CSV 파일 복사 및 추가 완료


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import os
import pandas as pd
from PIL import Image

In [ ]:
# csv img로 변환
!python generate_training_data.py -d ./data -fer C:/Users/USER/Desktop/big6/fer2013.csv -ferplus C:/Users/USER/Desktop/big6/fer2013new.csv